In [ ]:
from Detik import Detik #class news Scrapper
import mysql.connector

In [ ]:
obj = Detik()

In [4]:
links = obj.getIndeksLink([], 1, 'news', 'news')

page  1
https://news.detik.com/indeks/all/1?date=08/03/2018
page  2
https://news.detik.com/indeks/all/2?date=08/03/2018


In [5]:
detail = obj.getDetailBerita(links[10:13])

memasukkan berita id  4147543
memasukkan berita id  4147530
memasukkan berita id  4147515


In [6]:
detail

[{'subcategory': 'Berita',
  'id': 4147543,
  'category': 'news',
  'url': 'https://news.detik.com/berita/d-4147543/tonton-blak-blakan-menkumham-2-napi-kakap-ancam-yasonna',
  'pubdate': '2018-08-03 07:20:30',
  'author': 'Erwin Dariyanto',
  'title': 'Tonton Blak blakan Menkumham, 2 Napi Kakap Ancam Yasonna',
  'source': 'detik',
  'comments': 0,
  'tags': 'lapas sukamiskin',
  'images': 'https://akcdn.detik.net.id/community/media/visual/2018/07/02/fc066d61-8c8d-4a9a-99c8-f5566b206db6_169.png?w=780&q=90',
  'content': 'Jakarta -Penjara rupanya belum sepenuhnya membuat seorang terpidana jera dan berhenti melakukan pelanggaran hukum. Menteri Hukum dan Hak Asasi Manusia Yasonna Hamonangan Laoly mengaku mendapat ancaman dari dua orang narapidana yang mendekam di Lembaga Pemasyarakatan Sukamiskin, Bandung.Yasonna tak menyebut siapa kedua narapidana dimaksud. Tapi yang pasti ancaman itu muncul terkait dengan upaya dia bersama para stafnya membehani berbagai penyimpangan yang terjadi di sana

In [11]:
con = mysql.connector.connect(user='root', password='',
                              host='127.0.0.1',
                              database='news_db')
cursor = con.cursor()

In [9]:
for det in detail:
    add_article = ("INSERT INTO article"
              "(post_id, author, pubdate, category, subcategory, content, comments, images, title, tags, url, source) "
              "VALUES (%(id)s, %(author)s, %(pubdate)s, %(category)s, %(subcategory)s, %(content)s, %(comments)s, %(images)s, %(title)s, %(tags)s, %(url)s, %(source)s)")
    # Insert article
    cursor.execute(add_article, det)
    con.commit()

cursor.close()

In [10]:
import pandas as pd

In [16]:
df1 = pd.read_sql("Select * from article", con)

In [17]:
df1

,id,post_id,author,pubdate,category,subcategory,content,comments,images,title,tags,url,source
0,1,4147543,Erwin Dariyanto,2018-08-03 07:20:30,news,Berita,Jakarta -Penjara rupanya belum sepenuhnya memb...,0,https://akcdn.detik.net.id/community/media/vis...,"Tonton Blak blakan Menkumham, 2 Napi Kakap Anc...",lapas sukamiskin,https://news.detik.com/berita/d-4147543/tonton...,detik
1,2,4147530,Ahmad Bil Wahid,2018-08-03 07:16:57,news,Berita,"Jakarta -Badan Meteorologi, Klimiatologi dan G...",0,https://akcdn.detik.net.id/community/media/vis...,BMKG: Waspadai Geombang Tinggi 3 Hari ke Depan,"bmkg,gelombang tinggi,cuaca buruk",https://news.detik.com/berita/d-4147530/bmkg-w...,detik
2,3,4147515,Ahmad Bil Wahid,2018-08-03 07:01:44,news,Berita,Jakarta - Nama Ketua Majelis Syuro PKS Salim S...,0,https://akcdn.detik.net.id/community/media/vis...,Prabowo Dilema Pilih Salim Segaf atau AHY,"prabowo,ahy,salim segaf,pilpres 2019",https://news.detik.com/berita/d-4147515/prabow...,detik


In [18]:
con.close()